In [21]:
import os 
import sys 

cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

from functools import cache 
import numpy as np 
import pandas as pd 
import altair as alt 
from IPython.display import clear_output
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../../.env')

from utils_notebook.utils import ddf, load_subgraph, remove_prefix
from utils_notebook.vega import (
    output_chart, 
    apply_css, 
    stack_order_expr, 
    wide_to_longwide,
)
from utils_notebook.queries import adjust_precision, QueryManager
from utils_notebook.testing import validate_season_series
from utils_notebook.css import css_tooltip_timeseries_multi_colored
from utils_notebook.queries import QueryManager
from utils_notebook.vega import condition_union, output_chart

https://api.thegraph.com/subgraphs/name/cujowolf/beanstalk


In [2]:
sg: Subgrounds
bs: Subgraph
sg, bs = load_subgraph()

In [3]:
q = QueryManager(sg, bs) 

In [4]:
pool = bs.Query.metapoolOracles(first=100000, orderBy="season", orderDirection="asc")
df = sg.query_df(
    [
        pool.balanceA, 
        pool.balanceB, 
        pool.season, 
        pool.deltaB, 
        pool.timestamp, 
    ],
    pagination_strategy=ShallowStrategy
)
df = remove_prefix(df, "metapoolOracles_")
df.balanceA /= 10**6
df.balanceB /= 10**18
df.deltaB /= 10**6

In [5]:
# Reverse engineer pool reserves from quantites used in TWAP calculation. 
df['diff_a'] = (df.balanceA - df.balanceA.shift(1))
df['diff_b'] = (df.balanceB - df.balanceB.shift(1))
df['diff_timestamp'] = (df.timestamp - df.timestamp.shift(1))
df['reserves_3crv'] = df['diff_a'] / df.diff_timestamp
df['reserves_bean'] = df['diff_b'] / df.diff_timestamp
assert df.season.min() == 6076
df = df.iloc[1:,]
df.head()

,balanceA,balanceB,season,deltaB,timestamp,diff_a,diff_b,diff_timestamp,reserves_3crv,reserves_bean
1,1823002386736.906738,2030942247135.286865,6077,685226.464263,1659812464,46790827319.678223,50589125510.827393,3533.0,13243936.40523,14319027.883053
2,1869934826665.250244,2081743078278.517822,6078,694584.048577,1659816002,46932439928.343506,50800831143.230957,3538.0,13265245.881386,14358629.492151
3,1917706461458.546631,2133501113442.06958,6079,701977.691800,1659819600,47771634793.296387,51758035163.551758,3598.0,13277274.817481,14385223.780865
4,1965608261630.781006,2186128547794.177979,6080,803240.828322,1659823204,47901800172.234375,52627434352.108398,3604.0,13291287.506169,14602506.756967
5,2013881826521.260254,2239501123147.967285,6081,849932.384158,1659826833,48273564890.479248,53372575353.789307,3629.0,13302167.23353,14707240.38407


In [6]:
df_szns = q.query_seasons(extra_cols=['price'], where={"season_gte": 6074})[['season', 'price']]
df_szns = df_szns.rename(columns={"price": "price_bean"})
df_szns.head()

,season,price_bean
0,6074,1.022000
1,6075,1.070000
2,6076,1.050748
3,6077,1.051615
4,6078,1.051964


In [7]:
df = df.merge(df_szns, how="left", on="season")
df.head()

,balanceA,balanceB,season,deltaB,timestamp,diff_a,diff_b,diff_timestamp,reserves_3crv,reserves_bean,price_bean
0,1823002386736.906738,2030942247135.286865,6077,685226.464263,1659812464,46790827319.678223,50589125510.827393,3533.0,13243936.40523,14319027.883053,1.051615
1,1869934826665.250244,2081743078278.517822,6078,694584.048577,1659816002,46932439928.343506,50800831143.230957,3538.0,13265245.881386,14358629.492151,1.051964
2,1917706461458.546631,2133501113442.06958,6079,701977.691800,1659819600,47771634793.296387,51758035163.551758,3598.0,13277274.817481,14385223.780865,1.052462
3,1965608261630.781006,2186128547794.177979,6080,803240.828322,1659823204,47901800172.234375,52627434352.108398,3604.0,13291287.506169,14602506.756967,1.062113
4,2013881826521.260254,2239501123147.967285,6081,849932.384158,1659826833,48273564890.479248,53372575353.789307,3629.0,13302167.23353,14707240.38407,1.063713


In [8]:
# Approximation of pool TVL in $, since we can't compute this exactly without the price of 3Crv 
df['pool_tvl_usd'] = 2 * df.price_bean * (df.reserves_bean - df.deltaB)
df['bean_fraction'] = (df.price_bean * df.reserves_bean) / df.pool_tvl_usd
df['3crv_fraction'] = 1 - df.bean_fraction
df.tail()

,balanceA,balanceB,season,deltaB,timestamp,diff_a,diff_b,diff_timestamp,reserves_3crv,reserves_bean,price_bean,pool_tvl_usd,bean_fraction,3crv_fraction
1804,100661581843132.9375,98381880955302.265625,7881,-3268.318106,1666306811,50334400518.328125,49222125636.234375,3600.0,13981777.921758,13672812.676732,0.999766,27345761.58377,0.499881,0.500119
1805,100711915856777.71875,98431103080938.5,7882,-3214.369540,1666310411,50334013644.78125,49222125636.234375,3600.0,13981670.456884,13672812.676732,0.999996,27351944.684327,0.499882,0.500118
1806,100762225613009.578125,98480327689564.421875,7883,508.269218,1666314011,50309756231.859375,49224608625.921875,3600.0,13974932.286628,13673502.396089,1.000056,27347519.629085,0.500019,0.499981
1807,100812534952228.234375,98529553774335.78125,7884,775.938733,1666317611,50309339218.65625,49226084771.359375,3600.0,13974816.449627,13673912.436489,1.000057,27347831.733072,0.500028,0.499972
1808,100862855314785.359375,98578769077673.125,7885,-2285.573630,1666321211,50320362557.125,49215303337.34375,3600.0,13977878.48809,13670917.593707,0.999821,27341511.327939,0.499916,0.500084


In [9]:
df.timestamp = pd.to_datetime(df.timestamp, unit='s')
id_cols = ['timestamp']
value_cols = ['reserves_3crv', 'reserves_bean', 'deltaB', 'bean_fraction', '3crv_fraction', 'pool_tvl_usd']
df = df[id_cols + value_cols]
df = df.resample("D", on="timestamp").apply(lambda v: v.mean()).reset_index() 
df = wide_to_longwide(df, "timestamp", id_cols, value_cols)

In [22]:
selection_rule = alt.selection_single(
    name="sss", 
    fields=['timestamp'], nearest=True, on='mouseover', empty='none', clear='mouseout'
)

base = (
    alt.Chart(df)
    .encode(
        x=alt.X(
            "timestamp:O", 
            axis=alt.Axis(
                formatType="time", 
                ticks=False, 
                labelExpr="timeFormat(toDate(datum.value), '%b %e, %Y')", 
                labelOverlap=True, 
                labelSeparation=30, 
                labelPadding=5, 
                title='Date', 
                labelAngle=0, 
            )
        ), 
        color=alt.Color("variable:O", legend=None), 
        
    )
    .properties(width=500, height=250)
)

tooltip_formats = {
    "bean_fraction": ".1%", 
    "3crv_fraction": ".1%", 
    "pool_tvl_usd": "$,d"
}
rule = (
    # selection captures nearest timestamp (for current mouse position) 
    # tooltip rendered uses this data point (pivoted, so we have all data for this timestamp) 
    base
    .add_selection(selection_rule)
    .mark_rule(opacity=0)
    .encode(
        tooltip=(
            [alt.Tooltip(f'timestamp', timeUnit="yearmonthdate", title="date")] + 
            [alt.Tooltip(f'{m}', format=tooltip_formats.get(m, ",d")) for m in value_cols]
        ), 
    )
    
)
reserves = (
    base
    .transform_filter(condition_union("==", "|", ['reserves_3crv', 'reserves_bean']))
    .mark_line()
    .encode(
        y=alt.Y(
            "value:Q", 
            axis=alt.Axis(title="Reserves", format=".2~s"), 
            scale=alt.Scale(domainMin=12*1e6) # TODO: don't hardcode this
        )
    )
    .properties(title="Bean:3Crv Pool Reserves")
)
delta_b = (
    base
    .transform_filter("datum.variable == 'deltaB'")
    .mark_line()
    .encode(
        y=alt.Y(
            "value:Q", axis=alt.Axis(title="DeltaB", format=".2~s")
        )
    )
    .properties(title="Bean:3Crv Pool DeltaB")
)
balance = (
    base
    .transform_filter(condition_union("==", "|", ['bean_fraction', '3crv_fraction']))
    .mark_line()
    .encode(
        y=alt.Y(
            "value:Q", 
            axis=alt.Axis(title="Pool Balance", format="%"), 
            scale=alt.Scale(domainMin=.45, domainMax=.55), # TODO: don't hardcode this
        ), 
    )
    .properties(title="Bean:3Crv Pool Balance")
)
tvl = (
    base
    .transform_filter("datum.variable == 'pool_tvl_usd'")
    .mark_line()
    .encode(
        y=alt.Y(
            "value:Q", 
            axis=alt.Axis(title="Pool TVL", format="$.2~s"), 
        ), 
    )
    .properties(title="Bean:3Crv Pool TVL")
)
c = (
    (
        ((reserves + rule) & (delta_b + rule)) | 
        ((balance + rule) & (tvl + rule))
    )
    .resolve_scale(y="independent")
    .resolve_axis(y="independent")
)
c

/Users/ALEX/opt/anaconda3/envs/beanstalk/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [23]:
# TODO: Update tooltip css once we get that fixed on the frontend. 
output_chart(c)

/Users/ALEX/opt/anaconda3/envs/beanstalk/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


<IPython.core.display.JSON object>